In [1]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
"""
Convert "x y DATAHEX" lines to a C header with a uint8_t array.
- Input format example:
    # x y Data
    0 0 2F
    1 0 32
    ...
- Output: .h file with include guard, size macro, and aligned static const array.
"""

import os

# ---- Paths (edit as needed) ----
input_file_path  = "/content/drive/MyDrive/FSRCNN/text_files/feature_input_file/input_pixels.txt"
output_file_path = "/content/drive/MyDrive/FSRCNN/text_files/feature_input_file/frame1_data.h"

# ---- C symbols (edit as needed) ----
HEADER_GUARD = "FRAME1_DATA_H"
SIZE_MACRO   = "FRAME1_SIZE_BYTES"
ARRAY_NAME   = "frame1_data"
ALIGN_ATTR   = "__attribute__((aligned(64)))"  # GCC/Clang. For MSVC you'd use __declspec(align(64))

# ---- Read and collect bytes (keep original order; no reverse) ----
bytes_hex = []
with open(input_file_path, "r", encoding="utf-8") as f:
    for raw in f:
        line = raw.strip()
        if not line or line.startswith("#"):
            continue
        parts = line.split()
        if len(parts) < 3:
            # Skip malformed lines silently
            continue
        data_hex = parts[2]

        # Normalize to two-digit uppercase hex without '0x' prefix
        if data_hex.startswith(("0x", "0X")):
            data_hex = data_hex[2:]
        data_hex = data_hex.upper()
        if len(data_hex) == 1:
            data_hex = "0" + data_hex
        elif len(data_hex) > 2:
            # If longer than 2 (e.g., stray), keep the last two nibbles
            data_hex = data_hex[-2:]

        bytes_hex.append(data_hex)

count = len(bytes_hex)

# ---- Write header file ----
with open(output_file_path, "w", encoding="utf-8") as out:
    # Include guard and headers
    out.write(f"#ifndef {HEADER_GUARD}\n")
    out.write(f"#define {HEADER_GUARD}\n\n")
    out.write("#include <stdint.h>\n\n")
    out.write(f"#define {SIZE_MACRO} {count}\n\n")

    # Array definition (static const to avoid multiple definition across TU)
    out.write(f"static const uint8_t {ARRAY_NAME}[{SIZE_MACRO}] {ALIGN_ATTR} = {{\n")

    # Pretty-print: 16 values per line
    per_line = 16
    for i, hx in enumerate(bytes_hex, 1):
        sep = "\n" if (i % per_line == 0) else " "
        out.write(f"    0x{hx},{sep}")

    # Close array and guard
    if count % per_line != 0:
        out.write("\n")
    out.write("};\n\n")
    out.write(f"#endif /* {HEADER_GUARD} */\n")

print(f"✅ Wrote header: {output_file_path}")
print(f"   Bytes: {count}, Array: {ARRAY_NAME}, Size macro: {SIZE_MACRO}")


✅ Wrote header: /content/drive/MyDrive/FSRCNN/text_files/feature_input_file/frame1_data.h
   Bytes: 57600, Array: frame1_data, Size macro: FRAME1_SIZE_BYTES


In [2]:
import re

# ---- Paths ----
input_file_path = "/content/drive/MyDrive/FSRCNN/text_files/deconv_input_file/layer7_expand_output_hex_packed.txt"
output_file_path = "/content/drive/MyDrive/FSRCNN/text_files/deconv_input_file/frame1_data.c"

def is_comment_or_empty(line: str) -> bool:
    # Treat lines starting with '#' or empty/whitespace-only as comments
    s = line.strip()
    return (not s) or s.startswith('#')

def parse_hex_string_from_line(line: str) -> str:
    """
    Input line format example:
      "0 0 1C1E0A1E0100001910001504181E160D"
      "10 0 192201190608002000001F051A201217"
    We expect the last whitespace-separated token to be the hex string.
    """
    parts = line.strip().split()
    if len(parts) < 3:
        raise ValueError(f"Invalid data line (need x y HEXSTRING): {line}")
    hex_str = parts[-1]
    # Basic validation: even length and hex characters
    if len(hex_str) % 2 != 0:
        raise ValueError(f"Hex string must have even length: {hex_str}")
    if not re.fullmatch(r"[0-9A-Fa-f]+", hex_str):
        raise ValueError(f"Non-hex characters found: {hex_str}")
    return hex_str

# Collect bytes here
byte_list = []

with open(input_file_path, "r", encoding="utf-8") as infile:
    for raw in infile:
        if is_comment_or_empty(raw):
            continue
        hex_str = parse_hex_string_from_line(raw)

        # Reverse by byte pairs (2 hex chars per byte), e.g. "A1B2C3" -> ["C3","B2","A1"]
        for i in range(len(hex_str) - 2, -1, -2):
            b = hex_str[i:i+2]
            byte_list.append(f"0x{b.upper()}")

# Write C file
with open(output_file_path, "w", encoding="utf-8") as outfile:
    outfile.write("// Auto-generated from txt -> C array (byte-reversed per line)\n")
    outfile.write("// Input: bottom-up style per-line hex strings, reversed in 2-hex-digit steps\n")
    outfile.write("#include <stdint.h>\n\n")
    outfile.write(f"#define FRAME2_SIZE_BYTES {len(byte_list)}\n\n")
    outfile.write("const uint8_t frame2_data[FRAME2_SIZE_BYTES] __attribute__((aligned(64))) = {\n")

    # Pretty-print 16 bytes per line
    for i, token in enumerate(byte_list, start=1):
        outfile.write(f"  {token},")
        if i % 16 == 0:
            outfile.write("\n")
        else:
            outfile.write(" ")
    if len(byte_list) % 16 != 0:
        outfile.write("\n")
    outfile.write("};\n")

print(f"✅ Generated {output_file_path}")
print(f"   Total bytes: {len(byte_list)}")


✅ Generated /content/drive/MyDrive/FSRCNN/text_files/deconv_input_file/frame1_data.c
   Total bytes: 921600
